In [5]:
import krippendorff
import json


In [1]:
import json
import pingouin as pg
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# Carregar o JSON de um arquivo
with open('judge_evaluations.json', 'r') as file:
    data = json.load(file)

# Lista de aspectos a serem avaliados
all_aspects = [
    "organization_and_clarity", "use_of_examples", "argumentation", "coherence",
    "persuasion", "engagement", "adaptability", "preparation", "subject_mastery"
]

# Função para extrair a matriz
def extract_aspect_matrix(json_data, aspect):
    """
    Converte avaliações de um critério específico em uma matriz para cálculo de ICC e outras correlações.
    Mantém as notas numéricas.
    """
    evaluators = {}
    for entry in json_data:
        debate_id = entry["debate_id"]
        evaluator_id = entry["evaluator"]
        if debate_id not in evaluators:
            evaluators[debate_id] = {}
        for debater in entry["debaters"]:
            name = debater["name"]
            score = debater["evaluation_aspects"][aspect]
            if name not in evaluators[debate_id]:
                evaluators[debate_id][name] = {}
            evaluators[debate_id][name][evaluator_id] = score

    # Construir matriz para avaliação
    matrix = []
    for debate_id, debaters in evaluators.items():
        for name, scores in debaters.items():
            matrix.append([scores.get(evaluator, None) for evaluator in sorted(scores.keys())])
    return matrix

# Resultados agregados
results = {"ICC": [], "Pearson": [], "Spearman": []}

# Iterar por todos os aspectos e calcular métricas
for aspect in all_aspects:
    print(f"Calculando métricas para o aspecto: {aspect}")
    # Extrair matriz para o aspecto
    matrix = extract_aspect_matrix(data, aspect)
    
    # Transpor a matriz
    matrix = list(map(list, zip(*matrix)))
    
    # Preparar dados para o ICC
    df = pd.DataFrame(matrix)
    df_long = pd.melt(df, var_name="rater", value_name="rating")
    df_long["target"] = np.tile(df.index, len(df.columns))
    
    # Calcular ICC
    icc_result = pg.intraclass_corr(data=df_long, targets="target", raters="rater", ratings="rating")
    icc_mean = icc_result["ICC"].mean()
    results["ICC"].append(icc_mean)
    
    # Calcular correlação de Pearson
    pearson_corrs = []
    for i in range(len(matrix)):
        for j in range(i + 1, len(matrix)):
            corr, _ = pearsonr(matrix[i], matrix[j])
            pearson_corrs.append(corr)
    results["Pearson"].append(np.mean(pearson_corrs))
    
    # Calcular correlação de Spearman
    spearman_corrs = []
    for i in range(len(matrix)):
        for j in range(i + 1, len(matrix)):
            corr, _ = spearmanr(matrix[i], matrix[j])
            spearman_corrs.append(corr)
    results["Spearman"].append(np.mean(spearman_corrs))

# Calcular médias gerais
icc_overall = np.mean(results["ICC"])
pearson_overall = np.mean(results["Pearson"])
spearman_overall = np.mean(results["Spearman"])

# Exibir resultados
print("\nResultados por aspecto:")
for idx, aspect in enumerate(all_aspects):
    print(f"- {aspect}: ICC = {results['ICC'][idx]:.3f}, "
          f"Pearson = {results['Pearson'][idx]:.3f}, "
          f"Spearman = {results['Spearman'][idx]:.3f}")

print("\nMédias gerais:")
print(f"ICC médio: {icc_overall:.3f}")
print(f"Correlação de Pearson média: {pearson_overall:.3f}")
print(f"Correlação de Spearman média: {spearman_overall:.3f}")


Calculando métricas para o aspecto: organization_and_clarity
Calculando métricas para o aspecto: use_of_examples
Calculando métricas para o aspecto: argumentation
Calculando métricas para o aspecto: coherence
Calculando métricas para o aspecto: persuasion
Calculando métricas para o aspecto: engagement
Calculando métricas para o aspecto: adaptability
Calculando métricas para o aspecto: preparation
Calculando métricas para o aspecto: subject_mastery

Resultados por aspecto:
- organization_and_clarity: ICC = 0.545, Pearson = 0.467, Spearman = 0.486
- use_of_examples: ICC = 0.570, Pearson = 0.471, Spearman = 0.480
- argumentation: ICC = 0.502, Pearson = 0.472, Spearman = 0.460
- coherence: ICC = 0.574, Pearson = 0.386, Spearman = 0.405
- persuasion: ICC = 0.499, Pearson = 0.522, Spearman = 0.519
- engagement: ICC = 0.599, Pearson = 0.603, Spearman = 0.598
- adaptability: ICC = 0.579, Pearson = 0.439, Spearman = 0.447
- preparation: ICC = 0.613, Pearson = 0.484, Spearman = 0.511
- subject_m

In [4]:

def extract_aspect_matrix(json_data, aspect):
    evaluators = {}
    for entry in json_data:
        debate_id = entry["debate_id"]
        evaluator_id = entry["evaluator"]
        if debate_id not in evaluators:
            evaluators[debate_id] = {}
        for debater in entry["debaters"]:
            name = debater["name"]
            score = debater["evaluation_aspects"][aspect]
            if name not in evaluators[debate_id]:
                evaluators[debate_id][name] = {}
            evaluators[debate_id][name][evaluator_id] = score

    matrix = []
    for debate_id, debaters in evaluators.items():
        for name, scores in debaters.items():
            matrix.append([scores.get(evaluator, None) for evaluator in sorted(scores.keys())])
    return matrix

# Função para calcular ICC por tipo e aspecto
def calculate_icc_metrics(json_data, aspects):
    results = []

    for aspect in aspects:
        matrix = extract_aspect_matrix(json_data, aspect)
        matrix = list(map(list, zip(*matrix)))

        # Preparar os dados para ICC
        df = pd.DataFrame(matrix)
        df_long = pd.melt(df, var_name="rater", value_name="rating")
        df_long["target"] = np.tile(df.index, len(df.columns))

        # Calcular ICC
        icc_result = pg.intraclass_corr(data=df_long, targets="target", raters="rater", ratings="rating")

        # Extração dos valores específicos
        avg_absolute = icc_result.loc[icc_result["Type"] == "ICC1k", "ICC"].values[0]
        avg_random = icc_result.loc[icc_result["Type"] == "ICC2k", "ICC"].values[0]
        avg_fixed = icc_result.loc[icc_result["Type"] == "ICC3k", "ICC"].values[0]

        results.append({
            "aspect": aspect,
            "average_absolute": avg_absolute,
            "average_random": avg_random,
            "average_fixed": avg_fixed,
        })

    return pd.DataFrame(results)

# Calcular resultados (exemplo com dados fictícios no JSON "data")
# results_df = calculate_icc_metrics(data, aspects)
# print(results_df)

# Função para calcular médias gerais
def calculate_overall_means(results_df):
    mean_absolute = results_df["average_absolute"].mean()
    mean_random = results_df["average_random"].mean()
    mean_fixed = results_df["average_fixed"].mean()
    return {
        "mean_absolute": mean_absolute,
        "mean_random": mean_random,
        "mean_fixed": mean_fixed,
    }


aspects = [
    "organization_and_clarity",
    "use_of_examples",
    "argumentation",
    "coherence",
    "persuasion",
    "engagement",
    "adaptability",
    "preparation",
    "subject_mastery"
]

# Calcular ICCs para todos os aspectos
results_df = calculate_icc_metrics(data, aspects)

# Calcular médias gerais dos tipos de ICCs
overall_means = calculate_overall_means(results_df)
print("Médias gerais:", overall_means)

Médias gerais: {'mean_absolute': np.float64(0.9209276786609468), 'mean_random': np.float64(0.9243645434881258), 'mean_fixed': np.float64(0.9590270365514226)}


Os resultados indicam uma alta consistência entre os avaliadores quando consideramos as médias nos diferentes modelos de ICC:

Interpretação dos resultados:
Média do ICC (Average raters absolute): 0.921
Isso mostra que, em média, há uma excelente confiabilidade absoluta ao considerar as avaliações de vários avaliadores combinados.

Média do ICC (Average random raters): 0.924
Indica que a confiabilidade permanece alta mesmo quando os avaliadores são considerados como aleatoriamente selecionados.

Média do ICC (Average fixed raters): 0.959
Este valor ainda maior reflete uma altíssima consistência entre os avaliadores fixos ao combinar suas avaliações.

Conclusão:
Essas métricas sugerem que, ao considerar a média das avaliações de vários avaliadores, a consistência é muito alta, o que valida a confiabilidade do método de avaliação usado. Agora, você pode usar esses dados como uma base sólida para justificar as análises comparativas entre métodos ou validar a qualidade dos avaliadores.